In [141]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

accidentes = pd.read_csv("traffic_accidents_clean.csv",index_col="num_expediente")
accidentes

,fecha,hora_24h,fecha_hora,distrito,tipo_accidente,condiciones_meteorológicas,tipo_vehiculo,tipo_persona,rango_edad,sexo,lesividad,alcohol_test,drogas_test,coordenada_x_utm,coordenada_y_utm
num_expediente,,,,,,,,,,,,,,,
2018S017842,2019-02-04,09:10,2019-02-04 09:10:00,Centro,Colisión lateral,Despejado,Motocicleta > 125cc,Conductor,45 - 49,Hombre,Asistencia sanitaria sólo en el lugar del acci...,No,No,4.400680e+05,4.475679e+06
2018S017842,2019-02-04,09:10,2019-02-04 09:10:00,Centro,Colisión lateral,Despejado,Turismo,Conductor,30 - 34,Mujer,Asistencia sanitaria sólo en el lugar del acci...,No,No,4.400680e+05,4.475679e+06
2019S000001,2019-01-01,03:45,2019-01-01 03:45:00,Carabanchel,Alcance,No registrado,Furgoneta,Conductor,40 - 44,Hombre,Se desconoce,Si,No,4.391396e+05,4.470837e+06
2019S000001,2019-01-01,03:45,2019-01-01 03:45:00,Carabanchel,Alcance,No registrado,Turismo,Conductor,40 - 44,Mujer,Se desconoce,No,No,4.391396e+05,4.470837e+06
2019S000001,2019-01-01,03:45,2019-01-01 03:45:00,Carabanchel,Alcance,No registrado,Turismo,Conductor,45 - 49,Mujer,Se desconoce,No,No,4.391396e+05,4.470837e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023S040267,2023-12-31,21:15,2023-12-31 21:15:00,Villaverde,Colisión fronto-lateral,Despejado,Turismo,Pasajero,45 - 49,Mujer,Asistencia sanitaria sólo en el lugar del acci...,No,No,4.411526e+08,4.466350e+09
2023S040267,2023-12-31,21:15,2023-12-31 21:15:00,Villaverde,Colisión fronto-lateral,Despejado,Turismo,Pasajero,6 - 9,Hombre,Asistencia sanitaria sólo en el lugar del acci...,No,No,4.411526e+08,4.466350e+09
2023S040277,2023-12-29,09:35,2023-12-29 09:35:00,Centro,Alcance,Despejado,Motocicleta hasta 125cc,Conductor,45 - 49,Hombre,Se desconoce,No,No,4.395949e+08,4.473164e+09


In [153]:
# LIMPIEZA DE DATOS


accidentes.isna().sum()

accidentes['fecha'] = pd.to_datetime(accidentes['fecha'])

# Datos faltantes sustituir por la moda

accidentes['rango_edad'] = accidentes['rango_edad'].replace('Desconocido', np.nan)
moda_edad = accidentes['rango_edad'].mode()[0]
accidentes['rango_edad'] = accidentes['rango_edad'].fillna(moda_edad)

accidentes['sexo'] = accidentes['sexo'].replace('Desconocido', np.nan)
moda_sexo = accidentes['sexo'].mode()[0]
accidentes['sexo'] = accidentes['sexo'].fillna(moda_sexo)

accidentes['lesividad'] = accidentes['lesividad'].replace('Se desconoce', np.nan)
moda_lesividad = accidentes["lesividad"].mode()[0]

accidentes["lesividad"] = accidentes["lesividad"].fillna(moda_lesividad)

# Datos faltantes tipo float

media_coordenadaX = accidentes["coordenada_x_utm"].mean()
accidentes["coordenada_x_utm"] = accidentes["coordenada_x_utm"].fillna(media_coordenadaX)
media_coordenadaY = accidentes["coordenada_y_utm"].mean()
accidentes["coordenada_y_utm"] = accidentes["coordenada_y_utm"].fillna(media_coordenadaY)

# Datos no registrados en condiciones meteorologicas
indices_a_eliminar = accidentes[accidentes["condiciones_meteorológicas"] == "No registrado"].index
accidentes = accidentes.drop(indices_a_eliminar, axis=0)



In [155]:
# # Muertes por año

accidentes['año'] = accidentes['fecha'].dt.year
accidentes["mes"] = accidentes["fecha"].dt.month

# asistencia_sanitaria = accidentes.loc[:,"lesividad"].unique()

accidentes["muertes"] = accidentes.loc[:,"lesividad"] == "Fallecido 24 horas"

muerte_anual = accidentes.groupby("año")[["muertes"]].sum().reset_index()
muerte_mes = accidentes.groupby(["mes","año"])[["muertes"]].sum().reset_index()

total_muertes = accidentes.loc[:,"muertes"].sum()
total_muertes



np.int64(143)

In [222]:
def franja_horaria(hora):
    if 0 <= hora < 7:
        return "Madrugada"
    elif 7 <= hora < 12:
        return "Mañana"
    elif 12 <= hora < 19:
        return "Tarde"
    else:
        return "Noche"

# Extraer hora como float/int
accidentes['hora_int'] = accidentes['hora_24h'].str.split(':').str[0].astype(float)

# Crear la columna franja_horaria aplicando la función
accidentes['franja_horaria'] = accidentes['hora_int'].apply(franja_horaria)

# Agrupar muertes por año y franja horaria
muertes_por_franja = accidentes.groupby(['año', 'franja_horaria'])[['muertes']].sum().reset_index()

# Definir orden lógico de las franjas horarias
muertes_por_franja["franja_horaria"] = pd.Categorical(
    muertes_por_franja["franja_horaria"],
    ordered=True,
    categories=["Madrugada", "Mañana", "Tarde", "Noche"]
)

# Calcular total de muertes por año (para el porcentaje anual)
total_muertes_por_año = muertes_por_franja.groupby('año')['muertes'].transform('sum')

# Añadir columna con porcentaje de muertes respecto al total de cada año
muertes_por_franja['%muertes_hora_anual'] = (muertes_por_franja['muertes'] / total_muertes_por_año) * 100

# Obtener la franja horaria con más muertes por año (solo la primera si hay empate)
max_por_año = []

for año in muertes_por_franja['año'].unique():
    datos_año = muertes_por_franja[muertes_por_franja['año'] == año]
    max_row = datos_año.loc[datos_año['muertes'].idxmax()]
    max_por_año.append(max_row)

max_por_año = pd.DataFrame(max_por_año).reset_index(drop=True)

max_por_año

,año,franja_horaria,muertes,%muertes_hora_anual
0,2019,Noche,14,41.176471
1,2020,Noche,10,29.411765
2,2021,Tarde,10,37.037037
3,2022,Tarde,9,40.909091
4,2023,Mañana,8,30.769231


In [221]:
#  Año + Clima


# Total accidentes por clima
total_accidentes_clima = accidentes['condiciones_meteorológicas'].value_counts()

# Total muertes por clima
muertes_clima = accidentes.groupby('condiciones_meteorológicas')['muertes'].sum()

# Tasa de mortalidad por clima (%)
tasa_mortalidad_clima = (muertes_clima / total_accidentes_clima) * 100
tasa_mortalidad_clima = tasa_mortalidad_clima.sort_values(ascending=False)

tasa_mortalidad_clima




condiciones_meteorológicas
Nublado           0.123998
Despejado         0.075303
LLuvia intensa    0.069037
Lluvia débil      0.040920
Granizando        0.000000
Nevando           0.000000
dtype: float64

In [ ]:
# Año + Tipo de vía


Tasa de mortalidad por clima (%):
condiciones_meteorológicas
Nublado           0.123998
Despejado         0.075303
LLuvia intensa    0.069037
Lluvia débil      0.040920
Granizando        0.000000
Nevando           0.000000
dtype: float64

Tasa de mortalidad por franja horaria (%):
franja_horaria
Madrugada    0.142100
Noche        0.090022
Mañana       0.071633
Tarde        0.054955
dtype: float64


In [ ]:
# Año + Tipo de vehículo

In [ ]:
#  Año + Tipo de accidente